In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
pixels = 28*28
nums = 10 

x = tf.placeholder(tf.float32 ,[None,pixels],name="x")
y_ = tf.placeholder(tf.float32, [None,nums], name="y_")

In [8]:
def weight_variable(name,shape):
    W_init = tf.truncated_normal(shape,stddev=0.1)
    W = tf.Variable(W_init,name="W_"+name)
    return W

def bias_variable(name,size):
    b_init = tf.constant(0.1,shape=[size])
    b = tf.Variable(b_init,name="b_"+name)
    return b

def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

def max_pool(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

with tf.name_scope('conv1') as scope:
    W_conv1 = weight_variable('conv1',[5,5,1,32])
    b_conv1 = bias_variable('conv1',32)
    x_image = tf.reshape(x,[-1,28,28,1])
    h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)

with tf.name_scope('pool1') as scope:
    h_pool1 = max_pool(h_conv1)
    
with tf.name_scope('conv2') as scope:
    W_conv2 = weight_variable('conv2',[5,5,32,64])
    b_conv2 = bias_variable('conv2',64)
    h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
    
with tf.name_scope('pool2') as scope:
    h_pool2 = max_pool(h_conv2)
    
with tf.name_scope('fully_connected') as scope :
    n = 7*7*64
    W_fc = weight_variable('fc',[n,1024])
    b_fc = bias_variable('fc',1024)
    h_pool2_flat = tf.reshape(h_pool2,[-1,n])
    h_fc = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc)+b_fc)
    
with tf.name_scope('dropout') as scope:
    keep_prob = tf.placeholder(tf.float32)
    h_fc_drop = tf.nn.dropout(h_fc,keep_prob)

In [9]:
with tf.name_scope('readout') as scope:
    W_fc2 = weight_variable('fc2',[1024,10])
    b_fc2 = bias_variable('fc2', 10)
    y_conv =tf.nn.softmax(tf.matmul(h_fc_drop,W_fc2)+b_fc2)
    
with tf.name_scope('loss') as scope:
    cross_entoropy = -tf.reduce_sum(y_*tf.log(y_conv))

with tf.name_scope('training') as scope:
    optimizer = tf.train.AdamOptimizer(1e-4)
    train_step  = optimizer.minimize(cross_entoropy)

In [20]:
with tf.name_scope('predict') as scope:
    predict_step = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_,1))
    accuracy_step = tf.reduce_mean(tf.cast(predict_step,tf.float32))

def set_feed(images , labels ,prob):
    return {x:images, y_:labels, keep_prob:prob}

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    tw = tf.summary.FileWriter('log_dir',graph=sess.graph)
    
    test_fd = set_feed(mnist.test.images[0:1000,:],mnist.test.labels[0:1000,:],1)
    
    for step in range(10000):
        #batch = mnist.train.next_batch(10)
        
        test_image = mnist.test.images[0:1000,:]
        test_label = mnist.test.labels[0:1000,:]
        fd = set_feed(test_image,test_label,0.5)
        _,loss = sess.run([train_step,cross_entoropy],feed_dict= fd)
        if step % 100 == 0:
            acc =sess.run(accuracy_step,feed_dict=test_fd)
            print("step : ",step , "loss :" ,loss ,"acc=",acc)
    
    acc = sess.run(accuracy_step,feed_dict = test_fd)
    print("acc : " , acc)

step :  0 loss : 9343.227 acc= 0.195
step :  100 loss : 323.91174 acc= 0.97
step :  200 loss : 91.314804 acc= 0.999
step :  300 loss : 40.103695 acc= 1.0
step :  400 loss : 26.763325 acc= 1.0
step :  500 loss : 15.312971 acc= 1.0
step :  600 loss : 11.116772 acc= 1.0
step :  700 loss : 6.2018986 acc= 1.0
step :  800 loss : 6.8907986 acc= 1.0
step :  900 loss : 4.688984 acc= 1.0
step :  1000 loss : 3.0667486 acc= 1.0
step :  1100 loss : 4.353131 acc= 1.0
step :  1200 loss : 2.8431828 acc= 1.0
step :  1300 loss : 1.5442635 acc= 1.0
step :  1400 loss : 2.0593383 acc= 1.0
step :  1500 loss : 2.421784 acc= 1.0
step :  1600 loss : 0.90322936 acc= 1.0
step :  1700 loss : 2.866016 acc= 1.0


KeyboardInterrupt: 